In [1]:
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize
from ricerca import get_ordered_words, get_ordered_words_2
from def_overlap_funcs import filter_words

## Ricerca onomasiologica
Questo tipo di ricerca va a trovare una parola partendo dalle sue definizioni. Per fare l'esercizio viene usato il file definito in aula, nella tabella sotto sono riportate tutte le definizioni tokenizzate e pulite da stop words e punteggiatura.

In [2]:
data = pd.read_csv('TLN-definitions-23.csv', sep=';')
data = data.applymap(lambda s: filter_words(set(word_tokenize(s))))

data

,door,ladybug,pain,blurriness
0,"{closing, passage, two, divide, rooms, constru...","{red, small, flying, legs, six, typically, spo...","{distress, physical, feeling, mental}","{sight, focus}"
1,"{opening, closed, opened}","{luck, wings, many, black, insect, cultures, r...","{hurt, physical, emotional, feeling, bad}","{borders, absence, definite, shapelessness}"
2,"{open, enter, let, door, wall, hole, room, clo...","{color, red, orange, dots, fly, insect, black}","{conscious, felling, experience, beings, suffer}","{felt, something, clearly, sensation, see, ca,..."
3,"{access, one, usable, another, area}","{red, small, back, insect}","{describes, concept, suffering, living}","{lack, sharpness}"
4,"{access, allows, area, delimits, structure}","{small, round, flying, insect}","{physical, discomfort, feeling}","{lack, characteristic, clarity, precision}"
5,"{block, passage, object, pass, used, moved}","{colored, red, small, beatle, tendentially, fl...","{sensation, burning, unpleasant}","{eyes, blurring, sensation, see, everything, b..."
6,"{sometimes, handle, lock, requires, wood, sepa...","{small, back, control, luck, pest, used, black...","{felt, negative, body, sensation, damaged, cre...","{kind, goal, conceptual, certain, accurately, ..."
7,"{separate, object, wall, closed, opened, go, u...","{culture, red, small, dots, luck, brings, blac...","{pain, injury, caused, physical, emotional, sa...","{lack, image, eye, memory, vision, unfocused, ..."
8,"{access, another, something, place, opened, or...","{red, small, dots, insect, black}","{sentiment, sorrow, expresses}","{makes, clear, seeing, totally, bright, condit..."
9,"{access, room}","{red, small, coat, yellow, insect, dotted}","{exertion, sensation, physical, emotional, res...","{state, confusion, exertion, use, physical, al..."


### Lista genus
Vado a calcolare una lista di parole comuni per ognuno dei quattro termini. Questo mi permette di adottare la **strategia del genus**, ovvero far partire la ricerca del target da una parola molto usata nella sua definizione.
Vengono considerate solo le definizioni che hanno più di 4 parole. In realtà la lista dei genus non viene modificata molto

In [3]:
r = data.apply(lambda column: Counter(p for i in column for p in i if len(i) > 4).most_common())
r

door          [(object, 12), (room, 8), (access, 8), (allows...
ladybug       [(insect, 24), (red, 23), (black, 20), (small,...
pain          [(physical, 9), (emotional, 7), (feeling, 6), ...
blurriness    [(see, 7), (something, 5), (image, 5), (visual...
dtype: object

### Estrazione definizioni
Creo una lista con tutte le parole delle definizioni per un dato target che voglio trovare. La lista delle parole è già pulita da punteggiatura e stop words e tutte le parole compaiono una sola volta.

In [4]:
words_list_door = list(set().union(*data['door']))
words_list_bug = list(set().union(*data['ladybug']))
words_list_pain = list(set().union(*data['pain']))
words_list_blur = list(set().union(*data['blurriness']))

### Ricerca
Il metodo prende la lista di parole e il punteggio di similarità con la lista delle definizioni.
Il metodo prende tutti i synset di un genus e cicla su di essi, andando a calcolare la similarità tra il contesto del senso e il contesto delle definizioni. Ripete questa operazione su tutti i genus della lista in input.
Alla fine ordina la lista di $(senso,\ score\ similarità)$ risultato in modo *non crescente* per valore di similarità.

Nel metodo **get_ordered_words_2** l'intuizione è quella di non limitarsi agli iponimi del genus, ma provare a scendere anche lungo i figli. Viene mantenuto un limite per portare l'algoritmo a una terminazione veloce.
Per sfoltire si considerano solo i figli che hanno un valore di similarità associato sopra una certa soglia (la metà del valore massimo).
Per i **termini concreti** si è notato un miglioramento nella soluzione, in quanto i tra i sensi più alti ci sono gli elementi target delle descrizioni.
Per i **termini astratti** non ci sono stati cambiamenti degni di nota.
Questo approccio andrebbe rifinito per avere un *criterio d'arresto* più generico.

In una prima versione del metodo di base si provavano a prendere anche in considerazione i meronimi.

#### DOOR

In [5]:
get_ordered_words(r['door'], words_list_door)[0:10]

[('doorway', 9),
 ('domino', 6),
 ('opening', 6),
 ('screen_door', 6),
 ('pass-through', 6),
 ('cabin', 6),
 ('entrance', 5),
 ('transition', 5),
 ('room', 5),
 ('building', 5)]

In [6]:
get_ordered_words_2(r['door'], words_list_door, 4)[0:10]

[('doorway', 9),
 ('wall', 8),
 ('domino', 6),
 ('opening', 6),
 ('screen_door', 6),
 ('pass-through', 6),
 ('cabin', 6),
 ('door', 6),
 ('entrance', 5),
 ('transition', 5)]

#### LADYBUG

In [7]:
get_ordered_words(r['ladybug'], words_list_bug)[0:10]

[('nonsolid_color', 4),
 ('reddish_orange', 4),
 ('calendula', 4),
 ('dipterous_insect', 3),
 ('leaf_miner', 3),
 ('lepidopterous_insect', 3),
 ('psocopterous_insect', 3),
 ('scarlet', 3),
 ('worldly_possession', 3),
 ('horn_fly', 3)]

In [8]:
get_ordered_words_2(r['ladybug'], words_list_bug, 4)[0:10]

[('nonsolid_color', 4),
 ('reddish_orange', 4),
 ('calendula', 4),
 ('ladybug', 4),
 ('wax_begonia', 4),
 ('oncidium', 4),
 ('dipterous_insect', 3),
 ('leaf_miner', 3),
 ('lepidopterous_insect', 3),
 ('psocopterous_insect', 3)]

#### PAIN

In [9]:
get_ordered_words(r['pain'], words_list_pain)[0:10]

[('affection', 5),
 ('constriction', 5),
 ('pain', 4),
 ('actor', 4),
 ('toucher', 4),
 ('pinch', 4),
 ('wound', 4),
 ('agony', 4),
 ('discomfort', 4),
 ('photalgia', 4)]

In [10]:
get_ordered_words_2(r['pain'], words_list_pain, 4)[0:10]

[('affection', 5),
 ('constriction', 5),
 ('pain', 4),
 ('actor', 4),
 ('toucher', 4),
 ('pinch', 4),
 ('wound', 4),
 ('agony', 4),
 ('discomfort', 4),
 ('photalgia', 4)]

#### BLURRINESS

In [11]:
get_ordered_words(r['blurriness'], words_list_blur)[0:10]

[('placebo_effect', 5),
 ('vision', 5),
 ('like', 5),
 ('adapter', 5),
 ('look', 5),
 ('collage', 4),
 ('acuity', 4),
 ('naked_eye', 4),
 ('sounding_board', 4),
 ('user', 4)]

In [12]:
get_ordered_words_2(r['blurriness'], words_list_blur, 4)[0:10]

[('placebo_effect', 5),
 ('vision', 5),
 ('like', 5),
 ('adapter', 5),
 ('look', 5),
 ('collage', 4),
 ('acuity', 4),
 ('naked_eye', 4),
 ('sounding_board', 4),
 ('user', 4)]